In [1]:
!pip install mediapipe opencv-python numpy 

In [ ]:
import cv2
import mediapipe as mp
import json
import numpy as np
from threading import Thread

# MediaPipe Pose setup
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=False, model_complexity=0, enable_segmentation=False)
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

# Load the correct form JSON
with open('sitting.json', 'r') as json_file:
    correct_form = json.load(json_file)

# Thresholds
threshold = 0.4
highlight_threshold = 0.25


# VideoCapture with Threading for efficiency
class VideoCaptureThread:
    def __init__(self, src=0):
        self.cap = cv2.VideoCapture(src)
        self.ret = False
        self.frame = None
        self.thread = Thread(target=self.update, args=())
        self.thread.daemon = True
        self.thread.start()

    def update(self):
        while True:
            self.ret, self.frame = self.cap.read()

    def read(self):
        return self.ret, self.frame

    def release(self):
        self.cap.release()


# Function to calculate joint errors
def calculate_joint_errors(live_frame, correct_frame):
    joint_errors = []
    for i in range(len(live_frame)):
        error = np.sqrt(np.sum((np.array([live_frame[i]['x'], live_frame[i]['y'], live_frame[i]['z']]) -
                                np.array([correct_frame[i]['x'], correct_frame[i]['y'], correct_frame[i]['z']])) ** 2))
        joint_errors.append(error)
    return joint_errors


# Function to draw the skeleton with feedback
def draw_skeleton_with_feedback(frame, landmarks, joint_errors, threshold):
    height, width, _ = frame.shape
    faulty_landmarks = []
    connections = mp_pose.POSE_CONNECTIONS
    pixel_landmarks = [(int(l.x * width), int(l.y * height)) for l in landmarks]

    for idx, joint_error in enumerate(joint_errors):
        if joint_error > threshold:
            faulty_landmarks.append(idx)

    for connection in connections:
        start_idx, end_idx = connection
        start_point = pixel_landmarks[start_idx]
        end_point = pixel_landmarks[end_idx]

        if start_idx in faulty_landmarks or end_idx in faulty_landmarks:
            color = (0, 0, 255)  # Red for faulty connections
        else:
            color = (0, 255, 0)  # Green for correct connections

        cv2.line(frame, start_point, end_point, color, 3)

    for idx, (cx, cy) in enumerate(pixel_landmarks):
        color = (0, 255, 0) if idx not in faulty_landmarks else (0, 0, 255)
        cv2.circle(frame, (cx, cy), 5, color, -1)


# Main function
def main():
    cap = VideoCaptureThread(0)  # Initialize threaded video capture

    frame_skip = 3  # Process every 3rd frame
    frame_count = 0

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        frame_count += 1
        if frame_count % frame_skip != 0:
            continue  # Skip this frame

        # Resize frame to reduce processing load
        scale_factor = 0.5
        frame = cv2.resize(frame, (int(frame.shape[1] * scale_factor), int(frame.shape[0] * scale_factor)))

        image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose.process(image_rgb)

        if results.pose_landmarks:
            live_frame_landmarks = [{'x': landmark.x, 'y': landmark.y, 'z': landmark.z} for landmark in results.pose_landmarks.landmark]
            
            for correct_frame in correct_form:
                joint_errors = calculate_joint_errors(live_frame_landmarks, correct_frame)
                draw_skeleton_with_feedback(frame, results.pose_landmarks.landmark, joint_errors, threshold)

                overall_error = np.mean(joint_errors)
                feedback_text = "Pose matches!" if overall_error < threshold else f"Adjust your pose! Error: {overall_error:.4f}"
                cv2.putText(frame, feedback_text, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)

        # Display the frame (no need to enlarge the frame size now)
        cv2.imshow('Live Yoga Pose with Feedback', frame)

        # Quit on 'q' key press
        if cv2.waitKey(100) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()


if __name__ == "__main__":
    main()
